# Universidad del Valle de Guatemala  
## Facultad de Ingeniería  
### Departamento de Ciencias de la Computación  
#### Curso: CC3084 – Data Science  
#### Semestre II – 2025  

# Proyecto 1 – Obtención y Limpieza de Datos

### Autores:
- José Rodrigo Marchena – 22398  
- Andre Yatmian Jo – 22199  
- Héctor Daniel Penedo – 22217  
- Sofía Velásquez – 22049  

---

### Introducción

En este proyecto se trabaja con datos reales del Ministerio de Educación de Guatemala sobre establecimientos educativos a nivel diversificado. El objetivo principal es obtener, consolidar, limpiar y dejar listo este conjunto de datos para posteriores análisis, asegurando que el proceso de limpieza sea transparente, reproducible y bien documentado.

La limpieza se enfocará en eliminar errores comunes como inconsistencias tipográficas, duplicados, valores faltantes y variaciones no intencionales en nombres de instituciones. Las variables clave como nombre del establecimiento, dirección y teléfono recibirán especial atención, ya que son esenciales para identificar correctamente cada unidad educativa.

---

In [4]:
import pandas as pd
# Cargar el archivo CSV creado a partir del script de readfiles.py que unifica todos los xmls de establecimientos en un solo CSV
file_path = "./datos/establecimiento_all.csv"
df = pd.read_csv(file_path)

In [5]:
# Mostrar dimensiones generales
print("Dimensiones iniciales del dataset:")
print(f"Número de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")

# Visualizar primeras filas
df.head()

Dimensiones iniciales del dataset:
Número de filas: 6153
Número de columnas: 18


,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL,DEPARTMENT
0,16-01-0138-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO COBAN,KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8,77945104,PATRICIO NAJARRO ASENCIO,GUSTAVO ADOLFO SIERRA POP,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ
1,16-01-0139-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO PARTICULAR MIXTO VERAPAZ,KM 209.5 ENTRADA A LA CIUDAD,77367402,PATRICIO NAJARRO ASENCIO,GILMA DOLORES GUAY PAZ DE LEAL,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ
2,16-01-0140-46,16-031,ALTA VERAPAZ,COBAN,"COLEGIO ""LA INMACULADA""",7A. AVENIDA 11-109 ZONA 6,78232301,PATRICIO NAJARRO ASENCIO,VIRGINIA SOLANO SERRANO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ
3,16-01-0141-46,16-005,ALTA VERAPAZ,COBAN,ESCUELA NACIONAL DE CIENCIAS COMERCIALES,2A CALLE 11-10 ZONA 2,79514215,NORA LILIANA FIGUEROA HERNÁNDEZ,HÉCTOR ROLANDO CHUN POOU,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ
4,16-01-0142-46,16-005,ALTA VERAPAZ,COBAN,INSTITUTO NORMAL MIXTO DEL NORTE 'EMILIO ROSAL...,3A AVE 6-23 ZONA 11,79521468,NORA LILIANA FIGUEROA HERNÁNDEZ,VICTOR HUGO DOMÍNGUEZ REYES,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,BILINGUE,VESPERTINA,DIARIO(REGULAR),ALTA VERAPAZ,ALTA VERAPAZ


#### Análisis inicial

El dataset cuenta con **6,153 registros** y **18 columnas**. Cada fila representa un establecimiento educativo en un municipio y departamento específico de Guatemala.  
A continuación, se listan las variables que componen el dataset:

- `CODIGO`: Identificador único del establecimiento
- `DISTRITO`: Distrito escolar
- `DEPARTAMENTO`, `MUNICIPIO`: Ubicación geográfica
- `ESTABLECIMIENTO`: Nombre del establecimiento
- `DIRECCION`: Dirección física
- `TELEFONO`: Número telefónico de contacto
- `SUPERVISOR`, `DIRECTOR`: Autoridades del establecimiento
- `NIVEL`, `SECTOR`, `AREA`, `STATUS`: Clasificaciones administrativas
- `MODALIDAD`, `JORNADA`, `PLAN`: Características del modelo educativo
- `DEPARTAMENTAL`: Información adicional geográfica
- `DEPARTMENT`: Departamento de origen del archivo original

Este conjunto de datos presenta un nivel de **desorganización moderada**, con potenciales inconsistencias de formato, errores ortográficos y posibles duplicados.
